In [ ]:
import pandas as pd
import numpy as np
import pulp as pu
from tqdm import tqdm
import os

from bios_utils.problema import Problema

# Lectura de datos desde archivo

In [ ]:
# Archivo proporcionado por BIOS
bios_input_file = 'data/0_model_template_20240705.xlsm'

# Capacidad de descarga en puerto por día
cap_descarge = 5000000

## Parámetros generales

In [ ]:
problema = Problema(bios_input_file=bios_input_file, cap_carga_camion=34000)

# Creacion del Modelo de alcance de Objetivo

In [ ]:
from bios_utils.evitar_backorder_model import EvitarBackorder

In [ ]:
modelo_01 = EvitarBackorder(problema)

In [ ]:
modelo_01.solve()

##  Reporte Fase 1

In [ ]:
reporte_despachos = list()
for ingrediente in modelo_01.despachos_planta.keys():
    for planta in modelo_01.despachos_planta[ingrediente].keys():
        for periodo in modelo_01.despachos_planta[ingrediente][planta].keys():
            valor = modelo_01.despachos_planta[ingrediente][planta][periodo].varValue
            tiempo_proceso = problema.tiempos_proceso[planta][ingrediente]
            dato = {
                'variable': 'despacho a planta',
                'ingrediente': ingrediente,
                'planta': planta,
                'periodo': periodo,
                'valor': valor,
                'tiempo_recepcion': valor*tiempo_proceso
            }
            reporte_despachos.append(dato)

In [ ]:
reporte_inventario_puerto = list()
for ingrediente in modelo_01.inventario_puerto.keys():
    for periodo in modelo_01.inventario_puerto[ingrediente].keys():
        dato = {
            'variable': 'inventario en puerto',
            'ingrediente': ingrediente,
            'periodo': periodo,
            'valor': modelo_01.inventario_puerto[ingrediente][periodo].varValue
        }
        reporte_inventario_puerto.append(dato)

In [ ]:
reporte_inventario_planta = list()
for planta in modelo_01.inventario_planta.keys():
    for ingrediente in modelo_01.inventario_planta[planta].keys():
        for periodo in modelo_01.inventario_planta[planta][ingrediente]:
            dato = {
                'variable': 'inventario en planta',
                'planta': planta,
                'ingrediente': ingrediente,
                'periodo': periodo,
                'valor': modelo_01.inventario_planta[planta][ingrediente][periodo].varValue,
                'capacidad': modelo_01.capacidad_planta[planta][ingrediente],
                'consumo': modelo_01.consumo_proyectado[planta][ingrediente][periodo],
                'objetivo':modelo_01. objetivo_inventario[planta][ingrediente]
            }
            reporte_inventario_planta.append(dato)

## Fase 2
Dado que ya se tendrá un plan de recepcion de camiones en las plantas, la fase 2 asigna el invenatario en puerto a los camiones
a despachar, minimizando el costo de almacenamiento y transporte

## Parámetros

### Lista de importaciones

In [ ]:
importaciones = list()
for i in tqdm(range(cargas_despachables_df.shape[0])):
    inventario = int(cargas_despachables_df.iloc[i]['Inventario'])
    llegadas = int(cargas_despachables_df.iloc[i]['Llegada'])

    if inventario > 34000 or llegadas > 34000:

        index = (cargas_despachables_df.iloc[i]['Empresa'],
                 cargas_despachables_df.iloc[i]['Puerto'],
                 cargas_despachables_df.iloc[i]['Operador'],
                 cargas_despachables_df.iloc[i]['Ingrediente'],
                 cargas_despachables_df.iloc[i]['Importacion'])

        importaciones.append(index)

importaciones = list(set(importaciones))

### Llegadas a puerto

In [ ]:
transitos_a_puerto_df = cargas_despachables_df[cargas_despachables_df['Llegada'] > 0].copy(
)
transitos_a_puerto_df.set_index(
    ['Empresa', 'Puerto', 'Operador', 'Ingrediente', 'Importacion', 'Fecha'], inplace=True)

In [ ]:
llegadas_puerto = dict()
for i in tqdm(importaciones):
    llegadas_puerto[i] = dict()
    for periodo in periodos:
        i2 = tuple(list(i) + [periodo])

        if i2 in transitos_a_puerto_df.index:
            cantidad_puerto = int(transitos_a_puerto_df.loc[i2]['Llegada'])
        else:
            cantidad_puerto = 0
        llegadas_puerto[i][periodo] = cantidad_puerto

### Inventario inicial Puerto

In [ ]:
inventario_inicial_puerto_df = cargas_despachables_df[cargas_despachables_df['Inventario'] > 0].copy(
)
inventario_inicial_puerto_df.set_index(
    keys=['Empresa', 'Puerto', 'Operador', 'Ingrediente', 'Importacion'], inplace=True)

In [ ]:
inventario_inicial_puerto = dict()
for i in tqdm(importaciones):
    if i in inventario_inicial_puerto_df.index:
        cantidad = int(
            inventario_inicial_puerto_df.loc[i]['Inventario'])
    else:
        cantidad = 0
    inventario_inicial_puerto[i] = cantidad

### Costos de transporte

In [ ]:
columns = ['Empresa', 'Puerto', 'Operador',
           'Ingrediente', 'Importacion', 'valor_kg']
importaciones_df = cargas_despachables_df.groupby(
    columns)[[]].count().reset_index().copy()

In [ ]:
df = importaciones_df.rename(columns={'Empresa': 'Empresa_Origen'}).copy()

print(df.shape)

In [ ]:
# Cruzar con fechas de consumo
df = pd.merge(left=df,
              right=pd.DataFrame(periodos).rename(columns={0: 'Fecha'}),
              how='cross')
print(df.shape)

In [ ]:
temp = fletes_df.copy()
columns = ['puerto', 'operador', 'ingrediente']
temp = temp.melt(id_vars=columns, value_vars=temp.drop(
    columns=columns).columns, var_name='Planta', value_name='flete_kg')
temp['Empresa_Destino'] = temp['Planta'].map(empresas_dict)
temp['Flete_Camion'] = 34000*temp['flete_kg']
temp.drop(columns=['flete_kg'], inplace=True)
temp.rename(columns={'puerto': 'Puerto', 'operador': 'Operador',
            'ingrediente': 'Ingrediente'}, inplace=True)
print(temp.shape)

In [ ]:
# Cruzar con fletes
df = pd.merge(left=df,
              right=temp,
              left_on=['Puerto', 'Operador', 'Ingrediente'],
              right_on=['Puerto', 'Operador', 'Ingrediente'],
              how='left')
print(df.shape)

In [ ]:
costo_portuario_directo_df = costo_portuario_directo_df.rename(columns={'operador': 'Operador',
                                                                        'puerto': 'Puerto',
                                                                        'ingrediente': 'Ingrediente',
                                                                        'valor_kg': 'Directo'})

In [ ]:
# cruzar transitos con operacion portuaria de despacho directo
temp = pd.merge(left=transitos_a_puerto_df.reset_index().drop(columns=['Llegada', 'Camiones', 'valor_kg', 'Inventario']).rename(columns={'Empresa': 'Empresa_Origen'}),
                right=costo_portuario_directo_df,
                left_on=['Operador', 'Puerto', 'Ingrediente'],
                right_on=['Operador', 'Puerto', 'Ingrediente'],
                how='left')
# temp['Fecha'] = temp['Fecha'].apply(lambda x: x.strftime("%Y-%m-%d"))
temp['Directo'] = 34000*temp['Directo']
print(temp.shape)

In [ ]:
# Anexar costos portuarios por despacho directo
df = pd.merge(left=df,
              right=temp,
              left_on=['Empresa_Origen', 'Puerto', 'Operador',
                       'Ingrediente', 'Importacion', 'Fecha'],
              right_on=['Empresa_Origen', 'Puerto', 'Operador',
                        'Ingrediente', 'Importacion', 'Fecha'],
              how='left')
df['Directo'] = df['Directo'].fillna(0)

In [ ]:
intercompany_df = intercompany_df.melt(id_vars='origen', value_vars=empresas, var_name='Empresa_Destino',
                                       value_name='intercompany').rename(columns={'origen': 'Empresa_Origen'})

In [ ]:
# Intercompany
df = pd.merge(left=df,
              right=intercompany_df,
              left_on=['Empresa_Origen', 'Empresa_Destino'],
              right_on=['Empresa_Origen', 'Empresa_Destino'],
              how='left')

In [ ]:
df['Directo'].unique()

In [ ]:
# Calcular costo total despacho
df['CostoTotalCamion'] = df['Flete_Camion'] + df['Directo'] + \
    (34000*df['intercompany']*df['valor_kg'])

In [ ]:
costo_transporte_df = df.copy()

In [ ]:
costo_transporte_df.set_index(keys=['Empresa_Origen', 'Puerto', 'Operador',
                              'Ingrediente', 'Importacion', 'Planta', 'Fecha'], inplace=True)

In [ ]:
costo_transporte = dict()
for i in tqdm(importaciones):
    costo_transporte[i] = dict()
    for planta in plantas:
        costo_transporte[i][planta] = dict()
        for periodo in periodos:
            impo_index = tuple(list(i) + [planta, periodo])
            if impo_index in costo_transporte_df.index:
                costo = costo_transporte_df.loc[impo_index]['CostoTotalCamion']
                costo_transporte[i][planta][periodo] = costo

### Costo de almacenamiento

In [ ]:
df = costos_almacenamiento_df.set_index(
    ['empresa', 'puerto', 'operador', 'ingrediente', 'importacion', 'fecha_corte'])

In [ ]:
costo_almacenamiento = dict()
for i in tqdm(importaciones):
    costo_almacenamiento[i] = dict()
    for periodo in periodos:
        impo_index = tuple(list(i) + [periodo])
        if impo_index in df.index:
            costo = df.loc[impo_index]['valor_kg']
        else:
            costo = 0.0

        if not periodo in costo_almacenamiento[i].keys():
            costo_almacenamiento[i][periodo] = dict()
        costo_almacenamiento[i][periodo] = costo

In [ ]:
# Demanda de la planta
demanda_planta = dict()
for planta in tqdm(plantas):
    demanda_planta[planta] = dict()
    for ingrediente in ingredientes:
        demanda_planta[planta][ingrediente] = dict()
        for periodo in periodos[1:-2:]:
            demanda_planta[planta][ingrediente][periodo] = despachos_planta[ingrediente][planta][periodo].varValue

## Variables:

In [ ]:
# Inventario en puerto
var_inventario_puerto = dict()
for importacion in tqdm(importaciones):
    var_inventario_puerto[importacion] = dict()
    for periodo in periodos:
        var_name = f"inv_{'_'.join(list(importacion)).replace(' ','')}_{periodo}"
        var = pu.LpVariable(name=var_name, lowBound=0, cat=pu.LpContinuous)
        var_inventario_puerto[importacion][periodo] = var

In [ ]:
# Despachos hacia plantas
var_despachos = dict()
for importacion in tqdm(importaciones):
    var_despachos[importacion] = dict()
    for planta in plantas:
        var_despachos[importacion][planta] = dict()
        for periodo in periodos[1:-2:]:
            var_name = f'desp_{"_".join(importacion)}_{planta}_{periodo}'
            var = pu.LpVariable(name=var_name,
                                lowBound=0,
                                upBound=1000,
                                cat=pu.LpInteger)
            var_despachos[importacion][planta][periodo] = var

## Funcion Objetivo

Minimizar el costo de despacho y almacenamiento

$ \sum_{i}{\sum_{j}{CR_{i,j}X_{i,j}}} $


In [ ]:
# Costo de transporte
costo_transporte_fobj = [costo_transporte[i][j][t]*var_despachos[i][j][t]
                         for i in importaciones for j in plantas for t in periodos[1:-2:]]

In [ ]:
# Costo Almacenamiento
costo_almacenamiento_fobj = [34000*costo_almacenamiento[i][t] *
                             var_inventario_puerto[i][t] for i in importaciones for t in periodos]

In [ ]:
fobj = costo_transporte_fobj + costo_almacenamiento_fobj

## Restricciones

Cumplimiento de la demanda

$ \sum_{i}\sum_{j}\sum_{t}{X_{ijt}} >=  D_{jt}  $

In [ ]:
cumplimiento_demanda_rest = list()
for j in tqdm(plantas):
    for ingrediente in ingredientes:
        imp_list = [importaciones[i] for i in range(
            len(importaciones)) if importaciones[i][3] == ingrediente]
        for t in periodos[1:-2:]:
            left = pu.lpSum([var_despachos[i][j][t]
                            for i in importaciones if i[3] == ingrediente])
            right = demanda_planta[j][ingrediente][t]
            rest_name = f'cumplir_demanda_{ingrediente}_{j}_{t}'
            rest = (left == right, rest_name)
            cumplimiento_demanda_rest.append(rest)

In [ ]:
len(set([x for x in importaciones]))

Balance de inventario

$ I_{it} = I_{it-1} + A_{it} - \sum_{j}{X_{ijt}} \forall{i}, \forall {1>t>T-2}$

In [ ]:
balance_inventario_puerto_rest = list()
for i in tqdm(importaciones):

    # Generar inventario inicial como restriccion
    Iit = var_inventario_puerto[i][periodos[0]]
    if i in inventario_inicial_puerto.keys():
        Iit_1 = inventario_inicial_puerto[i]
    else:
        Iit_1 = 0
    Ait = llegadas_puerto[i][periodos[0]]
    rest_name = f"balance_inv_{'_'.join(i).replace(' ','_')}_{periodos[0]}"
    rest = (Iit == Iit_1 + Ait, rest_name)
    balance_inventario_puerto_rest.append(rest)

    # Balance de inventario con respecto al periodo anterior
    for t in periodos[1:-2:]:
        Iit = var_inventario_puerto[i][t]
        t_1 = periodos[periodos.index(t)-1]
        Iit_1 = var_inventario_puerto[i][t_1]
        Ait = llegadas_puerto[i][t]
        sum_des = [34000*var_despachos[i][j][t] for j in plantas]
        rest_name = f"balance_inv_{'_'.join(i).replace(' ','_')}_{t}"
        rest = (Iit == Iit_1 + Ait - pu.lpSum(sum_des), rest_name)
        balance_inventario_puerto_rest.append(rest)

In [ ]:
balance_inventario_puerto_rest[0][1]

## Resolver el model

In [ ]:
# Cantidad CPU habilitadas para trabajar
cpu_count = max(1, os.cpu_count()-1)

problema = pu.LpProblem(name='Bios_Solver_fase_2', sense=pu.LpMinimize)

# Agregando funcion objetivo
problema += pu.lpSum(fobj)

# Agregando balance de masa puerto
for rest in balance_inventario_puerto_rest:
    problema += rest

# cumplimiento de la demanda en la planta
for rest in cumplimiento_demanda_rest:
    problema += rest


t_limit_minutes = 5

print('------------------------------------')
print('cpu count', cpu_count)
print('ejecutando ', len(periodos), 'periodos')
engine_cbc = pu.PULP_CBC_CMD(
    timeLimit=60*t_limit_minutes,
    gapRel=0.05,
    warmStart=False,
    threads=cpu_count)

engine_glpk = pu.GLPK_CMD(
    mip=True,
    timeLimit=60*t_limit_minutes
)

problema.writeLP('model_2.lp')

problema.solve(solver=engine_cbc)

In [ ]:
pu.LpStatus[problema.status]

## Generar Reporte

In [ ]:
reporte_puerto = list()
for i in tqdm(importaciones):
    for t in periodos[:-2:]:
        dato = dict()
        dato['Empresa'] = i[0]
        dato['Puerto'] = i[1]
        dato['Operador'] = i[2]
        dato['ingrediente'] = i[3]
        dato['Importacion'] = i[4]
        dato['Fecha'] = t
        dato['Inventario'] = var_inventario_puerto[i][t].varValue
        dato['llegadas'] = llegadas_puerto[i][t]
        dato['Costo_Almacenamiento'] = int(costo_almacenamiento[i][t])
        dato['Costo_Total_Almacenamiento'] = dato['Inventario'] * \
            dato['Costo_Almacenamiento']
        reporte_puerto.append(dato)

reporte_puerto_df = pd.DataFrame(reporte_puerto)

In [ ]:
reporte_despachos = list()
for i in tqdm(importaciones):
    for j in plantas:
        for t in periodos[1:-2:]:
            dato = dict()
            dato['Empresa'] = i[0]
            dato['Puerto'] = i[1]
            dato['Operador'] = i[2]
            dato['ingrediente'] = i[3]
            dato['Importacion'] = i[4]
            dato['Fecha'] = t
            dato['Planta'] = j
            dato['Camiones_despachados'] = var_despachos[i][j][t].varValue
            dato['Costo_Transporte_camion'] = costo_transporte[i][j][t]
            dato['Costo_Transprote'] = dato['Camiones_despachados'] * \
                dato['Costo_Transporte_camion']
            reporte_despachos.append(dato)

reporte_despachos_df = pd.DataFrame(reporte_despachos)

In [ ]:
reporte_inventario_planta = list()
for planta in inventario_planta.keys():
    for ingrediente in inventario_planta[planta].keys():
        for periodo in inventario_planta[planta][ingrediente]:
            dato = {
                'variable': 'inventario en planta',
                'planta': planta,
                'ingrediente': ingrediente,
                'periodo': periodo,
                'valor': inventario_planta[planta][ingrediente][periodo].varValue,
                'capacidad': capacidad_planta[planta][ingrediente],
                'consumo': consumo_proyectado[planta][ingrediente][periodo],
                'backorder': backorder[planta][ingrediente][periodo].varValue,
                'objetivo': objetivo_inventario[planta][ingrediente]
            }
            reporte_inventario_planta.append(dato)

reporte_planta_df = pd.DataFrame(reporte_inventario_planta)

In [ ]:
with pd.ExcelWriter('reporte_final.xlsx') as writer:
    reporte_puerto_df.to_excel(
        writer, sheet_name='inventario_puerto', index=False)
    reporte_despachos_df.to_excel(writer, sheet_name='despachos', index=False)
    reporte_planta_df.to_excel(
        writer, sheet_name='inventario_planta', index=False)